In [16]:
# this is a test
# Core imports
import numpy as np
import heyoka as hk
import pykep as pk

# Sys imports
import time

# Plot imports
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

In [17]:
import math
from numpy.lib.function_base import append
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from numpy import linalg as la 

import dromo_const as const
from dromo_func import state2orb

In [22]:
# create heyoka variables
sigma, tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4 = hk.make_vars("sigma",
                                                                       "tau",
                                                                       "zeta1",
                                                                       "zeta2",
                                                                       "zeta3",
                                                                       "eta1",
                                                                       "eta2",
                                                                       "eta3",
                                                                       "eta4")


In [23]:
# Initial Conditions (dimensional)
# @t=t0*, (x0*, v0*) and (a0*, e0*, i0*, RAAN0*, omega0*, theta0*)
r0d = np.array([0.0, -5888.9727, -3400.0]) #km   (class 'numpy.ndarray')
v0d = np.array([10.691338, 0.0, 0.0])      #km/s
# perigee height should be 6800 km (correct!, as the minimum altitude comes out to be 428 km)
[a0d, e0d, i0d, RAAN0d, omega0d, theta0d] = state2orb(r0d, v0d, const.GMe)

# from the conversion comes out that a0* = 136,000 km
# and that the orbital and equatorial plane coincide, as i~=0 and RAAN is not defined

# Non-dimensionalise the ICs
r0 = r0d / a0d #[km/km] = [-] since the orbit is highly elliptical normalise with the initial semimajor axis
               # otherwise use r0 = r0d/la.norm(r0d)
v0 = v0d * math.sqrt(a0d/const.GMe)       #[-]
t0d = 0                                   #s
t0  = t0d / math.sqrt((a0d**3)/const.GMe) #[-]

# Tranform the non-dimensional ICs (r0, v0) in DROMO elements: (sigma; tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4)
h0 = np.cross(r0, v0)                    # 3-components vector
e0 = - r0/la.norm(r0) - np.cross(h0, v0) # 3-components vector
sigma_0 = 0
tau_0   = t0
zeta1_0 = la.norm(e0)
zeta2_0 = 0
zeta3_0 = 1/la.norm(h0)
eta1_0  = math.sin(i0d/2)*math.cos((RAAN0d-omega0d)/2)
eta2_0  = math.sin(i0d/2)*math.sin((RAAN0d-omega0d)/2)
eta3_0  = math.cos(i0d/2)*math.sin((RAAN0d-omega0d)/2)
eta4_0  = math.cos(i0d/2)*math.cos((RAAN0d-omega0d)/2)

S0 = [tau_0, zeta1_0, zeta2_0, zeta3_0, eta1_0, eta2_0, eta3_0, eta4_0]

In [28]:
# Auxiliary relationships
# s = hk.make_vars("s")
s = 1 + zeta1 * math.cos(sigma) + zeta2 * math.sin(sigma)

TypeError: must be real number, not heyoka.core.expression

In [29]:
# EOMs
dtaudsigma    = 1/(zeta3**3 * s**2)
dzeta1dsigma  = s * math.sin(sigma) * a_px   + (zeta1 + (1+s)*math.cos(sigma)) * a_py
dzeta2dsigma  = - s * math.cos(sigma) * a_px + (zeta2 + (1+s)*math.sin(sigma)) * a_py
dzeta3dsigma  = zeta3 * a_pz
deta1dsigma   = 1/2 * a_pz * (eta4 * math.cos(sigma) - eta3 * math.sin(sigma))
deta2dsigma   = 1/2 * a_pz * (eta3 * math.cos(sigma) + eta4 * math.sin(sigma))
deta3dsigma   = 1/2 * a_pz * (-eta2* math.cos(sigma) + eta1 * math.sin(sigma))
deta4dsigma   = 1/2 * a_pz * (-eta1* math.cos(sigma) - eta2 * math.sin(sigma))

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [9]:
# Define time

# tf = 288.12768941*24*3600            # s
tf = 2*np.pi * 50                      # roughly 50 orbits
delta_t = 1
n_steps = math.floor((tf-t0)/delta_t - 1)
# duration of integration in seconds
tspan = np.linspace(0, tf, n_steps)


In [14]:
print('Compiling the Taylor integrator ... (this is done only once)')
start_time = time.time()

ta = hk.taylor_adaptive(sys = [(tau,dtaudsigma),(zeta1,dzeta1dsigma),
                               (zeta2,dzeta2dsigma),(zeta3,dzeta3dsigma),
                               (eta1,deta1dsigma),(eta2,deta2dsigma),
                               (eta3,deta3dsigma),(eta4,deta4dsigma)],
                        state = S0,
                        time = sigma_0,
                        tol = 1e-16,
                        compact_mode = True)
end_time = time.time()

print('Done, in');
print("--- %s seconds ---" % (end_time - start_time))

print("\nHeyoka Taylor integrator:\n", ta)

Compiling the Taylor integrator ... (this is done only once)


NameError: name 'dtaudsigma' is not defined